In [2]:
import pandas as pd

import spacy
from spacy.tokens import DocBin

In [3]:
# Load dataset
alexa_rev = pd.read_json('..\\scrape_reviews\\alexa_reviews.json')
alexa_rev.head()

,name,stars,title,review
0,Parag,1,Very poor sound recognition.,It definitely has better speakers than Eco Dot...
1,Chinmaya D.,5,My first Smart Speaker,This was launched on 11th and I got the produc...
2,Vishnu,5,it’s amazing just read my review,The media could not be loaded. Such an amazin...
3,Rajiv p.,4,Stunner!,It's totally Paisa wasoolSpeaker is loudShe li...
4,Aritra Daspoddar,4,A real competition to Google Home mini,At first i was confused which one to buy - Goo...


In [5]:
alexa_rev.stars.value_counts(dropna=False)

5    2572
4    1028
1     633
3     498
2     269
Name: stars, dtype: int64

In [10]:
drop_index = alexa_rev[alexa_rev['stars']==3].index
alexa_rev.drop(drop_index, inplace=True)
alexa_rev.stars.value_counts(dropna=False)

5    2572
4    1028
1     633
2     269
Name: stars, dtype: int64

In [16]:
data = list(zip(alexa_rev['review'],alexa_rev['stars']))
data[0]

("It definitely has better speakers than Eco Dot Gen2 and Google Home Mini, but sound recognition is very poor compared to Google Home Mini or even previous Eco Dot. Maybe it has something to do with cost cutting and Amazon going with only 4 far field microphones instead of 7.But due to it's poor recognition, it takes a lot of time, sometimes even minutes and does nothing... Which definitely takes away the experience. Unfortunately I can't return it... But I think Google Home Mini is miles ahead and better with all the new updates to Google Assistant (Sleep assist and stuff).Update as on 27th April 2019: Software has gone from bad to worst. It plays certain songs again and again, doesn't let you skip. Recognition has also gone from bad to worst, you have to yell for Alexa to understand what you want it to do only for it to tell you it doesn't understand what you said. And expect no support from Amazon about eco devices, it's very bad.",
 1)

In [18]:
def make_docs(data):
    docs = []
    for doc, label in nlp.pipe(data, as_tuples=True):
        if label<3:
            doc.cats['POSITIVE'] = 0
            doc.cats['NEGATIVE'] = 1
        else:
            doc.cats['POSITIVE'] = 1
            doc.cats['NEGATIVE'] = 0
        docs.append(doc)
    return docs

nlp = spacy.load('en_core_web_sm')

In [26]:
test_size = 10
test_split = (len(data)*(100-test_size))//100

train_valid_data = data[:test_split]
test_data = data[test_split:]

valid_size = 30
valid_split = (len(train_valid_data)*(100-valid_size))//100

train_data = train_valid_data[:valid_split]
valid_data = train_valid_data[valid_split:]

print('Size of train data: ', len(train_data))
print('Size of valid data: ', len(valid_data))
print('Size of test data: ', len(test_data))

Size of train data:  2835
Size of valid data:  1216
Size of test data:  451


In [27]:
train_docs = make_docs(train_data)
doc_bin = DocBin(docs=train_docs)
doc_bin.to_disk('input\\train.spacy')

valid_docs = make_docs(valid_data)
doc_bin = DocBin(docs=valid_docs)
doc_bin.to_disk('input\\valid.spacy')